In [140]:
import yfinance
import locale
import os
import pandas as pd

In [ ]:
# SNP500 ETF: https://finance.yahoo.com/quote/IVV/

sandp500 = yfinance.download (tickers = "IVV", start = "2025-01-01", 
                              interval = "1d")
sandp500 = sandp500.reset_index()
sandp500.columns = sandp500.columns.droplevel(1) # 2. Spaltenebene wegschmeißen
sandp500

/var/folders/58/r64xvznn1tb799qqpxxyf0m00000gn/T/ipykernel_31821/3032877075.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  sandp500 = yfinance.download (tickers = "IVV", start = "2025-01-01",
[*********************100%***********************]  1 of 1 completed


Price,Date,Close,High,Low,Open,Volume
0,2025-01-02,583.806824,590.226722,579.712380,588.497523,9400400
1,2025-01-03,590.922363,591.737285,585.615515,586.678886,5867300
2,2025-01-06,594.589539,598.823129,592.731141,595.434304,4188000
3,2025-01-07,587.901245,596.845429,585.943431,596.567137,3810700
4,2025-01-08,588.517395,589.690070,584.333483,587.821724,5303600
...,...,...,...,...,...,...
126,2025-07-08,623.330017,625.070007,622.520020,624.320007,4685600
127,2025-07-09,627.030029,627.719971,623.890015,625.750000,3019800
128,2025-07-10,628.849976,629.869995,626.010010,627.179993,2556800
129,2025-07-11,626.440002,627.820007,624.520020,625.760010,2739200


### 1 Tag vor Liberation Day gekauft und bis 11.07. gehalten

In [251]:
investment = 100

sp500_april_first = sandp500[sandp500['Date'] == '2025-04-01']['Close'].item()
share = investment / sandp500[sandp500['Date'] == '2025-04-01']['Close'].item()

sp500_july_11 = sandp500[sandp500['Date'] == '2025-07-11']['Close'].item()

print(f"Investiert: {investment}")
print(f"Finaler Portfoliowert: {share * sp500_july_11}")
print(f"Gewinn: {round(sp500_july_11 * share - investment, 2)}")
print(f"+{round(((sp500_july_11 / sp500_april_first) - 1) * 100, 2)}%")

Investiert: 100
Finaler Portfoliowert: 111.46120531438214
Gewinn: 11.46
+11.46%


### Bei jedem >=2% Dip 2025 gekauft und bis 11.07. gehalten

In [254]:
# Vor 1 Tag vor Liberation Day gekauft und bis 11.07. gehalten
drop_threhshold = 0.02

sandp500_drops = sandp500.copy()
sandp500_drops['Close_change_pct'] = sandp500_drops['Close'].pct_change()
sandp500_drops['investment_share'] = investment / sandp500_drops['Close']
drop_days = sandp500_drops[sandp500_drops['Close_change_pct'] <= -drop_threhshold]
drop_days

Price,Date,Close,High,Low,Open,Volume,Close_change_pct,investment_share
44,2025-03-10,559.945740,568.671258,554.787924,566.753234,7935600,-0.026050,0.178589
58,2025-03-28,556.382690,567.109207,555.954036,566.341584,7466000,-0.020189,0.179732
62,2025-04-03,538.388916,548.756507,537.740915,545.905424,12111300,-0.047831,0.185739
63,2025-04-04,506.029938,526.605706,505.840557,524.482331,17517100,-0.060103,0.197617
67,2025-04-10,525.419373,534.261757,510.336464,533.015648,10186400,-0.035131,0.190324
71,2025-04-16,526.496033,535.876715,521.112817,532.467377,6763400,-0.022307,0.189935
73,2025-04-21,514.772644,522.428731,509.239902,522.020034,4588600,-0.023247,0.194261


In [255]:
total_investment = len(drop_days) * investment
total_shares = drop_days['investment_share'].sum()

print(f"Investiert: {total_investment}")
print(f"Finaler Portfoliowert: {total_shares * sp500_july_11}")
print(f"Gewinn: {round(total_shares * sp500_july_11 - total_investment, 2)}")
print(f"+{round(((total_shares * sp500_july_11 / total_investment) - 1) * 100, 2)}%")

Investiert: 700
Finaler Portfoliowert: 824.5183718939793
Gewinn: 124.52
+17.79%


### Bei jedem >=2% Dip 2025 gekauft und wieder verkauft bei Erholung

In [ ]:
recovery_days = []
sandp500['Date'] = pd.to_datetime(sandp500['Date'])
drop_days['Date'] = pd.to_datetime(drop_days['Date'])

for _, row in drop_days.iterrows():
    drop_date = row['Date']
    drop_idx = sandp500[sandp500['Date'] == drop_date].index[0]

    prev_row = sandp500.loc[drop_idx - 1]
    recovery_target = prev_row['Close']
    
    after = sandp500.loc[drop_idx + 1:]
    recovered = after[after['Close'] >= recovery_target]

    if not recovered.empty:
        recovery_days.append(recovered.iloc[0])
    else:
        recovered.append("")

In [266]:
df = drop_days.reset_index(drop=True)
accumulated = 100

for idx, row in df.iterrows():
    try:
        share = accumulated / row['Close']
        print(f"bought on {row['Date']} for {row['Close']} with {accumulated}, share: {share}")

        accumulated = share * recovery_days[idx]['Close']
        print(f"sold on {recovery_days[idx]['Date']} for {recovery_days[idx]['Close']}, new balance: {accumulated}")
        print()
    except:
        continue

print()
print()
print(f"Investiert: {investment}")
print(f"Gewinn: {round(accumulated - investment, 2)}")
print(f"+{round(((accumulated/ investment) - 1) * 100, 2)}%")

bought on 2025-03-10 00:00:00 for 559.9457397460938 with 100, share: 0.17858873262495897
sold on 2025-03-24 00:00:00 for 574.9248046875, new balance: 102.6750922237927

bought on 2025-03-28 00:00:00 for 556.3826904296875 with 102.6750922237927, share: 0.1845404143405288
sold on 2025-05-12 00:00:00 for 583.8868408203125, new balance: 107.75071953296286

bought on 2025-04-03 00:00:00 for 538.388916015625 with 107.75071953296286, share: 0.20013547145505453
sold on 2025-05-02 00:00:00 for 567.5678100585938, new balance: 113.5904512487895

bought on 2025-04-04 00:00:00 for 506.0299377441406 with 113.5904512487895, share: 0.22447377670019047
sold on 2025-04-09 00:00:00 for 544.5496826171875, new balance: 122.23712385797015

bought on 2025-04-10 00:00:00 for 525.4193725585938 with 122.23712385797015, share: 0.2326467774926561
sold on 2025-04-24 00:00:00 for 547.5103759765625, new balance: 127.37652461473982

bought on 2025-04-16 00:00:00 for 526.4960327148438 with 127.37652461473982, share: 0